In [59]:
import onnx
from onnx import helper
from qonnx.core.modelwrapper import ModelWrapper
model = ModelWrapper.load("../tinyyolo_infershapes.onnx")
split_node = "Mul_97"

model.find_direct_successors(split_node)

ModuleNotFoundError: No module named 'qonnx'